In [59]:
'''
Challenge link
https://preppindata.blogspot.com/2023/01/2023-week-4-new-customers.html 
'''

import pandas as pd
import numpy as np
import datetime

In [60]:
#  We want to stack the tables on top of one another, since they have the same fields in each sheet.
df = pd.concat(pd.read_excel('New Customers.xlsx', sheet_name=None), ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2970 entries, 0 to 2969
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            2970 non-null   int64 
 1   Joining Day   2970 non-null   int64 
 2   Demographic   2436 non-null   object
 3   Value         2970 non-null   object
 4   Demographiic  258 non-null    object
 5   Demagraphic   276 non-null    object
dtypes: int64(2), object(4)
memory usage: 139.3+ KB


In [61]:
cust_df = pd.ExcelFile('New Customers.xlsx')
df_list = []
for sheet in cust_df.sheet_names:
    df = cust_df.parse(sheet)
    # df = df.replace(np.nan, '', regex=True)
    # df['Demographic'] = [x+y+z for x,y,z in zip(df['Demographic'],df['Demographiic'],df['Demagraphic'])]
    # df = df.drop(['Demographiic', 'Demagraphic'], axis=1)
    df['Month'] = str(sheet)
    df_list.append(df)

In [62]:
df_list[0].head(10)

,ID,Joining Day,Demographic,Value,Month
0,490910,3,Ethnicity,White,January
1,490910,3,Date of Birth,5/23/1981,January
2,490910,3,Account Type,Basic,January
3,369221,18,Ethnicity,Black,January
4,369221,18,Date of Birth,3/4/2019,January
5,369221,18,Account Type,Platinum,January
6,568991,16,Ethnicity,White,January
7,568991,16,Date of Birth,4/27/1979,January
8,568991,16,Account Type,Gold,January
9,191607,9,Ethnicity,White,January


In [63]:
df = pd.concat(df_list, ignore_index=True)
df.head(5)

,ID,Joining Day,Demographic,Value,Month,Demographiic,Demagraphic
0,490910,3,Ethnicity,White,January,NaN,NaN
1,490910,3,Date of Birth,5/23/1981,January,NaN,NaN
2,490910,3,Account Type,Basic,January,NaN,NaN
3,369221,18,Ethnicity,Black,January,NaN,NaN
4,369221,18,Date of Birth,3/4/2019,January,NaN,NaN


In [64]:
# Some of the fields aren't matching up as we'd expect, due to differences in spelling. Merge these fields together
df = df.replace(np.nan, '', regex=True)
df['Demographic'] = [x+y+z for x,y,z in zip(df['Demographic'],df['Demographiic'],df['Demagraphic'])]
df = df.drop(['Demographiic', 'Demagraphic'], axis=1)
df.head(10)

,ID,Joining Day,Demographic,Value,Month
0,490910,3,Ethnicity,White,January
1,490910,3,Date of Birth,5/23/1981,January
2,490910,3,Account Type,Basic,January
3,369221,18,Ethnicity,Black,January
4,369221,18,Date of Birth,3/4/2019,January
5,369221,18,Account Type,Platinum,January
6,568991,16,Ethnicity,White,January
7,568991,16,Date of Birth,4/27/1979,January
8,568991,16,Account Type,Gold,January
9,191607,9,Ethnicity,White,January


In [65]:
def getdate(day,mon):
    month_number = datetime.datetime.strptime(mon, '%B').month

    return f'{day}/{month_number}/2023'

df['Joining date'] = ''
for i in range(len(df)):
    df.loc[i,'Joining date'] = getdate(df.loc[i,'Joining Day'], df.loc[i,'Month'])
df.head(5)

,ID,Joining Day,Demographic,Value,Month,Joining date
0,490910,3,Ethnicity,White,January,3/1/2023
1,490910,3,Date of Birth,5/23/1981,January,3/1/2023
2,490910,3,Account Type,Basic,January,3/1/2023
3,369221,18,Ethnicity,Black,January,18/1/2023
4,369221,18,Date of Birth,3/4/2019,January,18/1/2023


In [66]:
df['Joining date'] = df['Joining date'].astype('datetime64[ns]')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2970 entries, 0 to 2969
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ID            2970 non-null   int64         
 1   Joining Day   2970 non-null   int64         
 2   Demographic   2970 non-null   object        
 3   Value         2970 non-null   object        
 4   Month         2970 non-null   object        
 5   Joining date  2970 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 139.3+ KB


C:\Users\mfram\AppData\Local\Temp\ipykernel_16628\2582729407.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['Joining date'] = df['Joining date'].astype('datetime64[ns]')


In [67]:
df.head(5)

,ID,Joining Day,Demographic,Value,Month,Joining date
0,490910,3,Ethnicity,White,January,2023-03-01
1,490910,3,Date of Birth,5/23/1981,January,2023-03-01
2,490910,3,Account Type,Basic,January,2023-03-01
3,369221,18,Ethnicity,Black,January,2023-01-18
4,369221,18,Date of Birth,3/4/2019,January,2023-01-18


In [68]:
df = df.drop(['Joining Day', 'Month'], axis=1)
df.head(5)

,ID,Demographic,Value,Joining date
0,490910,Ethnicity,White,2023-03-01
1,490910,Date of Birth,5/23/1981,2023-03-01
2,490910,Account Type,Basic,2023-03-01
3,369221,Ethnicity,Black,2023-01-18
4,369221,Date of Birth,3/4/2019,2023-01-18


In [69]:
# Now we want to reshape our data so we have a field for each demographic, for each new customer
cust_df = pd.DataFrame(columns=['ID', 'Ethnicity' , 'Date of Birth', 'Account Type', 'Joining date'])
cust_df.head(2)

,ID,Ethnicity,Date of Birth,Account Type,Joining date


In [70]:
for i in range(0,len(df),3):
    cust_df.loc[i,'ID'] = df.loc[i,'ID']
    cust_df.loc[i,'Ethnicity'] = df.loc[i,'Value']
    cust_df.loc[i,'Date of Birth'] = df.loc[i+1,'Value']
    cust_df.loc[i,'Account Type'] = df.loc[i+2,'Value']
    cust_df.loc[i,'Joining date'] = df.loc[i,'Joining date']

cust_df['Joining date'] = cust_df['Joining date'].astype('datetime64[ns]')
cust_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 990 entries, 0 to 2967
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             990 non-null    object        
 1   Ethnicity      990 non-null    object        
 2   Date of Birth  990 non-null    object        
 3   Account Type   990 non-null    object        
 4   Joining date   990 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 78.7+ KB


In [71]:
# Remove duplicates, If a customer appears multiple times take their earliest joining date

cust_df = cust_df.sort_values(by='Joining date')
cust_df.head(5)

,ID,Ethnicity,Date of Birth,Account Type,Joining date
15,863243,Other,12/30/2007,Platinum,2023-01-01
18,893948,White,9/1/2013,Gold,2023-01-01
255,627428,White,11/5/2018,Platinum,2023-01-02
297,598787,Other,9/11/1962,Basic,2023-01-02
585,248371,Asian,12/30/1979,Gold,2023-01-03


In [72]:
cust_df = cust_df.drop_duplicates('ID', keep='first')
cust_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 989 entries, 15 to 2811
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             989 non-null    object        
 1   Ethnicity      989 non-null    object        
 2   Date of Birth  989 non-null    object        
 3   Account Type   989 non-null    object        
 4   Joining date   989 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 46.4+ KB


In [ ]:
cust_df.to_csv('output.csv' , index=False)